<a href="https://colab.research.google.com/github/ehsanpour1/IBM/blob/master/Final_IBM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Most important thing for establishing a great restaurant is choose true location. For choose
true location you need to consider many parameters. First parameter is nearby restaurant to
purpose society.
In this study we want to investigate about establishing a Filipino restaurant in a big city and my
case study city is Toronto.
First group of my purpose society is Filipino and then other Asians, then I prefer to choose the
locations of the restaurant nearby of this two groups.
Other important parameter is household income and we should choose a location that average
of household income is enough four getting a food from restaurants.

# Data

To answer this question, following data will be used:
- Population and Ethnic of each neighborhood (Toronto Census: https://open.toronto.ca
/dataset/neighbourhood-profiles/)
- List of Toronto neighborhood (Wikipedia :https://en.wikipedia.org/wiki/List_of_postal_c
odes_of_Canada:_M)
- Household income of each neighborhood (Toronto Census: https://open.toronto.ca/)
–Toronto geospatial data (https://cocl.us/Geospatial_data)
- Number of Chinese restaurant (Foursquare API) – Google

# Methodology

First all data such as neighborhood, postal code, Filipino population, income and, latitude, longitude …
combined. The statistical data are plot on Toronto map, which can give an initial impression of areas
having highest Chinese population, and highest average income. Foursquare API was used to find
the restaurant count.
K-Mean clustering algorithms are used to analyses a Filipino Population, restaurant count and
people with Asian origin as the purpose society.
The result help for find best place to run a new Filipino restaurant plan.

In [7]:
import pandas as pd
import numpy as np
from itertools import *
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
import folium 
import json
from pandas.io.json import json_normalize
import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')

'en_US.UTF-8'

In [8]:
# get the post code data from wikipedia
df_toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
df_toronto.replace("Not assigned", np.nan, inplace = True)
df_toronto.dropna(subset=["Borough"], axis=0, inplace=True)
df_toronto.reset_index(drop=True, inplace=True)
df_toronto["Neighbourhood"].replace(np.nan, df_toronto["Borough"], inplace = True)
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [9]:
# collect statistics data about Filipino population and average income of each neighbourhood
filename = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/ef0239b1-832b-4d0b-a1f3-4153e53b189e?format=csv'
df_statistics = pd.read_csv(filename, thousands=',')
df_statistics

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,...,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,...,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,...,NIA,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,NIA,NIA,NIA,No Designation,No Designation,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,NIA,NIA,No Designation,No Designation,NIA,No Designation,NIA,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695","15,873","25,797","21,396","13,154","23,236","6,577","29,960","22,291","21,737","7,727","14,257","9,266","11,499","17,757","11,669","9,955","10,968","13,362","31,340","26,984","16,472","15,935","14,133","9,666","17,180","27,051","25,003","36,625","35,052",...,"10,803","10,529","22,246","14,974","20,923","46,496","10,070","9,941","16,724","21,849","27,876","17,812","24,623","25,051","27,446","15,683","21,567","10,360","21,108","16,556","7,607","17,510","65,913","27,392","33,312","26,274","17,992","11,098","27,917","50,434","16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918","15,434","19,348","17,671","13,530","23,185","6,488","27,876","21,856","22,057","7,763","14,302","8,713","11,563","17,787","12,053","9,851","10,487","13,093","28,349","24,770","14,612","15,703","13,743","9,444","16,712","26,739","24,363","34,631","34,659",...,"10,007","10,488","22,267","15,050","20,631","45,912"

In [10]:

df_statistics = df_statistics.loc[(df_statistics.Characteristic == 'Neighbourhood Number')|
                          (df_statistics.Characteristic == ' Asian origins')|
                          (df_statistics.Characteristic == ' Filipino')|
                          (df_statistics.Characteristic == 'Total income: Average amount ($)')]

df_statistics= df_statistics.drop(['_id', 'Category','Topic', 'Data Source', 'City of Toronto'],axis= 1)
df_statistics.rename({'Characteristic':'index'}, axis = 1, inplace = True)
df_statistics.set_index('index', inplace = True)
df_statistics = df_statistics.transpose()
df_statistics = df_statistics.reset_index() 
df_statistics.rename(columns = {'index':'Neighbourhood', ' Asian origins':'Asian origins', ' Filipino':'Filipino'}, inplace = True)
df_statistics['Neighbourhood Number'] = df_statistics["Neighbourhood Number"].astype("float")

df_statistics[['Asian origins', 'Filipino', 'Total income: Average amount ($)' ]]=df_statistics[['Asian origins', 'Filipino', 'Total income: Average amount ($)' ]].applymap(atof)

df_statistics.head()
print(df_statistics.dtypes)


index
Neighbourhood                        object
Neighbourhood Number                float64
Asian origins                       float64
Filipino                            float64
Total income: Average amount ($)    float64
dtype: object


In [11]:
# table of top ten neighbourhoods has most total average income 
df_statistics.sort_values (by = "Total income: Average amount ($)", ascending = False, axis=0, inplace = True)
df_statistics.head()

index,Neighbourhood,Neighbourhood Number,Asian origins,Filipino,Total income: Average amount ($)
16,Bridle Path-Sunnybrook-York Mills,41.0,3005.0,75.0,308010.0
104,Rosedale-Moore Park,98.0,3485.0,220.0,207903.0
44,Forest Hill South,101.0,1880.0,405.0,204521.0
69,Lawrence Park South,103.0,2510.0,285.0,169203.0
21,Casa Loma,96.0,1480.0,90.0,165047.0


In [12]:
# map of each neighbourhood's average income
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = "tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)


toronto_geo ='/content/simple.geojson'
toronto_map.choropleth( geo_data = toronto_geo, data = df_statistics, columns = ['Neighbourhood Number', 'Total income: Average amount ($)'],key_on = 'feature.properties.hoodnum',
    fill_color = 'YlOrRd',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Average Income',
    reset = True
    
)
toronto_map

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [13]:
# map of each neighbourhood's Filipino population
toronto_map_pop = folium.Map(location=[latitude, longitude], zoom_start=10)
toronto_map_pop.choropleth(
    geo_data = toronto_geo,
    data = df_statistics,
    columns = ['Neighbourhood Number', 'Filipino'],
    key_on = 'feature.properties.hoodnum',
    fill_color = 'PuBuGn',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = 'Filipino Population',
    reset = True
    
)
toronto_map_pop

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [14]:
# Find corresponsing postcode of each neighbourhood from wikipedia table. 
# For top 30 Filipino population, if not found in wikipedia, mannually add data from google
df_statistics["Postcode"] = np.nan
df_statistics['Postcode'] = df_statistics["Postcode"].astype("str")
df_statistics['Neighbourhood'] = df_statistics['Neighbourhood'].str.replace('-', '|')
for index, row in df_statistics.iterrows():
        postcode = df_toronto[df_toronto['Neighbourhood'].str.contains(row['Neighbourhood'])].Postcode
        if len(postcode):
            df_statistics.at[index, 'Postcode'] = postcode.iloc[0]
        else:
            row['Neighbourhood'].replace("|", " ")
            lt = list(row['Neighbourhood'].split(" "))
            postcode = df_toronto[(df_toronto['Neighbourhood'].isin(lt))].Postcode
            if len(postcode):
                df_statistics.at[index, 'Postcode'] = postcode.iloc[0]
        


df_statistics.sort_values(by = "Filipino", ascending = False, axis=0, inplace = True)
df_statistics.at[122, 'Postcode'] = 'M5P'
df_statistics.at[6, 'Postcode'] = 'M5G'
df_statistics.at[30, 'Postcode'] = 'M2J'
df_statistics.at[98, 'Postcode'] = 'M1W'
df_statistics.at[111, 'Postcode'] = 'M2P'
df_statistics.at[11, 'Postcode'] = 'M1P'
df_statistics = df_statistics[df_statistics.Postcode != 'nan']
df_statistics.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


index,Neighbourhood,Neighbourhood Number,Asian origins,Filipino,Total income: Average amount ($),Postcode
73,Malvern,132.0,28455.0,5095.0,29573.0,M1B
33,Downsview|Roding|CFB,26.0,11365.0,4790.0,34168.0,M3H
86,Newtonbrook West,36.0,14075.0,4475.0,34904.0,M2M
105,Rouge,131.0,28705.0,4365.0,39556.0,M1B
132,Woburn,137.0,32850.0,4345.0,30878.0,M1G
39,Englemount|Lawrence,32.0,6685.0,4090.0,48134.0,M6A
31,Dorset Park,126.0,15965.0,3660.0,31692.0,M1P
24,Clairlea|Birchmount,120.0,12705.0,3250.0,36232.0,M1K
11,Bendale,127.0,17890.0,3045.0,33256.0,M1P
61,Kennedy Park,124.0,9150.0,2750.0,30974.0,M1K


In [15]:
# form a table group by postcode 
df_statistics_gp = df_statistics.groupby('Postcode').agg({'Asian origins':'sum','Filipino':'sum','Total income: Average amount ($)':'mean'})
df_statistics_gp.head(10)

,Asian origins,Filipino,Total income: Average amount ($)
Postcode,,,
M1B,57160.0,9460.0,34564.500000
M1C,7175.0,880.0,40972.000000
M1E,21485.0,5015.0,39605.666667
M1G,32850.0,4345.0,30878.000000
M1J,8120.0,1040.0,32913.000000
M1K,29645.0,8535.0,32863.000000
M1L,7305.0,700.0,26793.000000
M1M,9895.0,1715.0,49539.000000
M1P,33855.0,6705.0,32474.000000


In [0]:
#combine neighboufhood postcode data with geospatial data
df_toronto["Neighbourhood"] = df_toronto["Neighbourhood"].astype("str")
df_toronto_clean = df_toronto.groupby(['Postcode','Borough'])['Neighbourhood'].apply(", ".join)
df_toronto_clean = df_toronto_clean.reset_index()
df_toronto_clean =df_toronto_clean.drop_duplicates()
df_position = pd.read_csv("https://cocl.us/Geospatial_data")
df_position = df_position.rename({'Postal Code': 'Postcode'}, axis = 1)
df_toronto_full = pd.merge(df_toronto_clean, df_position, on="Postcode", how="inner")

In [17]:
#combine statistics data and geospatial data
df_toronto_full = pd.merge(df_toronto_full, df_statistics_gp, on='Postcode', how = 'inner')
df_toronto_full.sort_values(by = "Filipino", ascending = False, axis = 0, inplace = True)
df_toronto_full.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($)
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,57160.0,9460.0,34564.500000
5,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,29645.0,8535.0,32863.000000
23,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,16555.0,6820.0,40052.000000
8,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,33855.0,6705.0,32474.000000
11,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,107920.0,6030.0,33871.400000
17,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,62290.0,6000.0,41814.000000
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,21485.0,5015.0,39605.666667
3,M1G,Scarborough,Woburn,43.770992,-79.216917,32850.0,4345.0,30878.000000
40,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,6685.0,4090.0,48134.000000
14,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,28930.0,3625.0,36869.000000


In [0]:
# write a function to collect venue data from foursquare by location data

def getNearbystatistics(names, query, latitudes, longitudes, radius=3000):
    
    statistics_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&categoryId={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            query,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        statistics_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_statistics = pd.DataFrame([item for statistics_list in statistics_list for item in statistics_list])
    nearby_statistics.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                   'statistics', 
                  ' statistics Latitude', 
                  ' statistics Longitude', 
                  ' statistics Category']
    
    return(nearby_statistics)

In [0]:
CLIENT_ID = '0NFSEYBHZISORKI30P1UAOQPV4RJGZLJV5RKSSHAYTYV0YNL' # your Foursquare ID
CLIENT_SECRET = 'KUKV1R5NLBM5ONZFNN304FDKNI2LSGCKAOCK4FVFBNZJUZ1R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [0]:
LIMIT = 500
df_toronto_statistics = getNearbystatistics(names = df_toronto_full['Neighbourhood'], 
                                    query = '4bf58dd8d48988d145941735',
                                  latitudes = df_toronto_full['Latitude'],
                                  longitudes = df_toronto_full['Longitude'])

In [21]:
df_toronto_statistics.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,statistics,statistics Latitude,statistics Longitude,statistics Category
0,"Rouge, Malvern",43.806686,-79.194353,Charley's Exotic Cuisine,43.800982,-79.200233,Chinese Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Toronto Jaffray Chinese Alliance Church,43.809241,-79.168471,Church
2,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Mandarin Buffet,43.730482,-79.279064,Chinese Restaurant
3,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Chung Moi,43.732328,-79.269327,Chinese Restaurant
4,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Spicy Dragon,43.749219,-79.275082,Asian Restaurant


In [22]:
#Table 7: Toronto Neighbourhood sorted by Filipino restaurant count
# find the total count of Filipino restaurant of each area
df_toronto_ChR = df_toronto_statistics.groupby('Neighbourhood')['statistics'].count().reset_index()

df_toronto_ChR.sort_values(by = "statistics", ascending = False, axis = 0, inplace = True)
df_toronto_ChR.rename({'statistics':'Filipino Restaurant Count'}, axis = 1, inplace = True)
df_toronto_ChR.head(10)

,Neighbourhood,Filipino Restaurant Count
12,"Chinatown, Grange Park, Kensington Market",100
48,"The Annex, North Midtown, Yorkville",100
39,"Little Portugal, Trinity",100
9,"Cabbagetown, St. James Town",100
13,Church and Wellesley,100
11,Central Bay Street,100
36,L'Amoreaux West,92
8,"CN Tower, Bathurst Quay, Island airport, Harbo...",87
1,"Agincourt North, L'Amoreaux East, Milliken, St...",85
0,Agincourt,79


In [23]:
#Table 8: Full table of Toronto Neighbourhood with Postcode, statistics, and restaurant count
# merge the restaurant count with statistics data
df_tor = pd.merge(df_toronto_full, df_toronto_ChR, on="Neighbourhood", how="outer")
df_tor.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,57160.0,9460.0,34564.500000,2
1,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,29645.0,8535.0,32863.000000,12
2,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,16555.0,6820.0,40052.000000,11
3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,33855.0,6705.0,32474.000000,16
4,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,107920.0,6030.0,33871.400000,85
5,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,62290.0,6000.0,41814.000000,19
6,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,21485.0,5015.0,39605.666667,3
7,M1G,Scarborough,Woburn,43.770992,-79.216917,32850.0,4345.0,30878.000000,8
8,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,6685.0,4090.0,48134.000000,8
9,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,28930.0,3625.0,36869.000000,28


In [0]:
#Use K-Means clustering algrithm to analyze the neighbourhood, the clustering number is set to 5.

kclusters = 5
df_tor.sort_values(by = "Filipino", ascending = False, axis = 0, inplace = True)
df_tor_clustering = df_tor.drop(['Postcode','Borough','Neighbourhood','Latitude','Longitude'], axis = 1)
df_tor_clustering.head()
#run k-means clustering
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_tor_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]

In [25]:
#Table 9: Toronto Neighbourhood with cluster labels
df_tor.sort_values(by = "Filipino", ascending = False, axis = 0, inplace = True)
df_tor.insert(0, 'Cluster Labels', kmeans.labels_)
df_tor.head(10)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
0,3,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,57160.0,9460.0,34564.500000,2
1,0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,29645.0,8535.0,32863.000000,12
2,0,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,16555.0,6820.0,40052.000000,11
3,0,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,33855.0,6705.0,32474.000000,16
4,3,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,107920.0,6030.0,33871.400000,85
5,3,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,62290.0,6000.0,41814.000000,19
6,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,21485.0,5015.0,39605.666667,3
7,0,M1G,Scarborough,Woburn,43.770992,-79.216917,32850.0,4345.0,30878.000000,8
8,0,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,6685.0,4090.0,48134.000000,8
9,0,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,28930.0,3625.0,36869.000000,28


# Discussion


Check 5 clusters independently. Cluster 0 has ralatively low average income, however not bad Filipino population. Cluster 1 has middle range of income from 50k to 80k. Cluster 2 has highest average income and middle amount of Filipino population. Cluster 3 has loweast average income. Cluster 4 has middle high range of income, and middle range of Filipino population. Map the clusters with different color, which gives an idea of eather cluster's location.

In [26]:
#Table 10: 5 Clusters data serperately
df_tor.dtypes
df_tor_cl0 = df_tor[df_tor['Cluster Labels'] == 0]
df_tor_cl0.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
1,0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,29645.0,8535.0,32863.000000,12
2,0,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,16555.0,6820.0,40052.000000,11
3,0,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,33855.0,6705.0,32474.000000,16
6,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,21485.0,5015.0,39605.666667,3
7,0,M1G,Scarborough,Woburn,43.770992,-79.216917,32850.0,4345.0,30878.000000,8


In [27]:
df_tor_cl1 = df_tor[df_tor['Cluster Labels'] == 1]
df_tor_cl1.head(10)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
59,1,M2L,North York,"Silver Hills, York Mills",43.75749,-79.374714,3005.0,75.0,308010.0,9


In [28]:
df_tor_cl2 = df_tor[df_tor['Cluster Labels'] == 2]
df_tor_cl2

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
16,2,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,5210.0,1880.0,123077.0,7
44,2,M2P,North York,York Mills West,43.752758,-79.400049,8615.0,435.0,100516.0,12
53,2,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3485.0,220.0,207903.0,23
54,2,M4G,East York,Leaside,43.709060,-79.363452,2740.0,215.0,125564.0,10


In [29]:
df_tor_cl3 = df_tor[df_tor['Cluster Labels'] == 3]
df_tor_cl3.head(10)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
0,3,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,57160.0,9460.0,34564.5,2
4,3,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,107920.0,6030.0,33871.4,85
5,3,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,62290.0,6000.0,41814.0,19


In [30]:
df_tor_cl4 = df_tor[df_tor['Cluster Labels'] == 4]
df_tor_cl4

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count
10,4,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,28080.0,3330.0,77849.500000,12
12,4,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,17550.0,2495.0,66281.333333,5
17,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,20830.0,1765.0,61487.200000,24
22,4,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,3020.0,1540.0,65274.000000,10
26,4,M3B,North York,Don Mills North,43.745906,-79.352188,12025.0,1280.0,67757.000000,9
34,4,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,8255.0,850.0,70623.000000,87
39,4,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,3845.0,545.0,64140.000000,6
41,4,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,2440.0,510.0,63012.000000,100
43,4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,15040.0,455.0,56526.000000,100
46,4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,6485.0,435.0,112766.000000,100


In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighbourhood'], df_tor['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Figure 3: Divide Toronto neighbourhood to 5 clusters based on the population and income

Table 11: Explanation of Clusters

In [32]:
data = [['cluster 0', 'low income, high population', 'Purple'], ['cluster 1', 'middle income', 'Blue'], ['cluster 2', 'high income, middle low population', 'Turquoise'], ['cluster 3', 'low income', 'Orange'],['cluster 4', 'middle high income, middle population', 'Red'] ]
df = pd.DataFrame(data, columns = ['Cluster', 'Meaning', 'Color'])
df

,Cluster,Meaning,Color
0,cluster 0,"low income, high population",Purple
1,cluster 1,middle income,Blue
2,cluster 2,"high income, middle low population",Turquoise
3,cluster 3,low income,Orange
4,cluster 4,"middle high income, middle population",Red


In [33]:

df_tor_review = df_tor[(df_tor['Cluster Labels'] == 4)|(df_tor['Cluster Labels'] == 2)]
df_tor_review.insert(10, 'Restaurant/Person', df_tor_review['Filipino Restaurant Count']/df_tor_review['Filipino'])
df_tor_review.sort_values(by = 'Restaurant/Person', ascending = True, axis = 0, inplace = True)
df_tor_review

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count,Restaurant/Person
12,4,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,17550.0,2495.0,66281.333333,5,0.002004
10,4,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,28080.0,3330.0,77849.500000,12,0.003604
16,2,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,5210.0,1880.0,123077.000000,7,0.003723
22,4,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,3020.0,1540.0,65274.000000,10,0.006494
26,4,M3B,North York,Don Mills North,43.745906,-79.352188,12025.0,1280.0,67757.000000,9,0.007031
39,4,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,3845.0,545.0,64140.000000,6,0.011009
17,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,20830.0,1765.0,61487.200000,24,0.013598
56,4,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262,1350.0,170.0,71888.000000,3,0.017647
49,4,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,3400.0,345.0,71204.000000,7,0.020290
44,2,M2P,North York,York Mills West,43.752758,-79.400049,8615.0,435.0,100516.000000,12,0.027586


From Table 12, it seems neighbourhood "Cloverdale, Islington, Martin Grove, Princess" with postcode "M9B" has the relatively low Filipino restaurant density and meanwhile has relatively high average income. The next two neighbourhoods are "Forest Hill North, Forest Hill West" and "Bedford Park, Lawrence Manor East". To confirm this result, all the neighbourhood is checked together with Restaurant per person ratio added.

Table 13: Full toronto neighbourhood data with Restaurant per person

In [0]:
df_tor.insert(10, 'Restaurant/Person', (df_tor['Filipino Restaurant Count']/df_tor['Filipino']))

In [35]:
df_tor.sort_values(by = "Restaurant/Person", ascending = True, axis = 0, inplace = True)
df_tor.head(5)

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count,Restaurant/Person
0,3,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,57160.0,9460.0,34564.500000,2,0.000211
6,0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,21485.0,5015.0,39605.666667,3,0.000598
1,0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,29645.0,8535.0,32863.000000,12,0.001406
2,0,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,16555.0,6820.0,40052.000000,11,0.001613
7,0,M1G,Scarborough,Woburn,43.770992,-79.216917,32850.0,4345.0,30878.000000,8,0.001841


Narrow down data with some threshold setting. Eg: Restaurant/Person < 0.007 and Average income > 100k.

Table 14 Neighbourhood has low density of Filipino restaurant but high income

In [36]:
df_tor_rate = df_tor[(df_tor['Restaurant/Person'] < 0.007) & (df_tor['Total income: Average amount ($)'] > 100000)]
df_tor_rate

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude,Asian origins,Filipino,Total income: Average amount ($),Filipino Restaurant Count,Restaurant/Person
16,2,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,5210.0,1880.0,123077.0,7,0.003723


The result is same as what we investicated from Clusters. From Figure 4, we can see the location of these top three neighbourhoods.

In [37]:
map_target = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor_rate['Latitude'], df_tor_rate['Longitude'], df_tor_rate['Neighbourhood'], df_tor_rate['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color='Black',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_target)
       
map_target

Figure 4: Neighbourhoods good to open a new Filipino restaurant



# Conclusion

In this study I try to consider the most important parameters such as locations , population, average income , and nearby to others Filllipino restaurant.
Of course , it's so challenging but I try to choose best locations to earn best result.

Brooke Avenue and neighbourhoods are best place for running a Filipino restaurant in toronto.